In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.inspection import partial_dependence
from mlp_keras import MLP
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# Load data, specify locations

folder = '../report/NO_OS/postop'
file = '/data.csv'

data = pd.read_csv(folder + file)

X, y = data.iloc[:,:-1], data.iloc[:, -1]
feature_names, target_names = data.columns[:-1].to_list(), data.columns[-1:].to_list()

In [4]:
# Split the data to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [5]:
# Load and train the model

is_model_nn = True

# RF
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
accuracy_score(y_test, model.predict(X_test))


# # NN
# model = MLP(folder + '/data_norm.csv')

# early_stopping_callback = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=20, restore_best_weights=True)
# rlr_callback = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss', factor=0.2, patience=5, min_lr=0.0000001)

# model.train(
#     9,
#     3,
#     'SGD',
#     0.01,
#     [early_stopping_callback, rlr_callback],
#     dropout=0.24,
#     save_folder=None,
#     oversampling=False,
#     test=False
# )

0.9064748201438849

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from PyALE import ale

df = pd.read_csv(folder + '/data_norm.csv').iloc[:, :-1]

for i, f in enumerate(df.columns.to_list()):
    plt.figure(figsize=(30, 13))
    ale_eff = ale(
        X=df,
        model=model,
        feature=[f],
        grid_size=10,
        include_CI=True
    )

    plt.savefig('tmp.png')


In [6]:
import shap

shap_explainer = shap.KernelExplainer(model, X_train.iloc[:100])
shap_values = shap_explainer.shap_values(X_test.iloc[[0]], check_additivity=False)
shap.summary_plot(shap_values, X_test.iloc[[0]])

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


Provided model function fails when applied to the provided data set.


TypeError: 'RandomForestClassifier' object is not callable

In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

def custom_scoring(est, X, y_true):
    y_pred = np.argmax(est.predict(X), axis=1)
    return accuracy_score(y_true.to_numpy(), y_pred)

if is_model_nn:
    results = permutation_importance(mlp.trained_model, X_train, y_train, n_repeats=10, random_state=42, scoring=custom_scoring)
else:
    results = permutation_importance(mlp.trained_model, X_train, y_train, n_repeats=10, random_state=42)

importance = results.importances_mean

for i, v in enumerate(importance):
    print(f'Feature {i}: {v:.5f}')

plt.bar(X_test.columns, importance)
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.show()

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

for c in y_train.unique():
    fig, ax = plt.subplots(figsize=(22, 15))
    pdp = PartialDependenceDisplay.from_estimator(mlp, X_train, features=[i for i in range(len(X_train.columns))], target=c, method='brute', ax=ax)